In [ ]:
from transformers import XLMRobertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup, AutoTokenizer
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

device = torch.device("cuda")

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

train_data = tokenizer(
    df_train['premise'].tolist(),
    df_train['hypothesis'].tolist(),
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors='pt',
    add_special_tokens=True
)

train_data['labels'] = torch.tensor(df_train['label'].tolist())

test_data = tokenizer(
    df_test['premise'].tolist(),
    df_test['hypothesis'].tolist(),
    truncation=True,
    padding="max_length",
    max_length=512,
    return_tensors='pt',
    add_special_tokens=True
)

input_ids_train = train_data['input_ids']
attention_mask_train = train_data['attention_mask']
labels_train = train_data['labels']

input_ids_test = test_data['input_ids']
attention_mask_test = test_data['attention_mask']

In [ ]:
class TrainDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

train_dataset = TrainDataset(input_ids_train, attention_mask_train, labels_train)


train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

In [ ]:
model = XLMRobertaForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=3
).to(device)

optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    weight_decay=0.01
)

In [ ]:
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")

In [ ]:
class TestDataset(Dataset):
    def __init__(self, input_ids, attention_mask):
        self.input_ids = input_ids
        self.attention_mask = attention_mask

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx]
        }

test_dataset = TestDataset(input_ids_test, attention_mask_test)

test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=2
)

In [ ]:
model.eval()
y_pred = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predicted = torch.argmax(outputs.logits, dim=1)
        y_pred.extend(predicted.cpu().numpy())

y_pred = np.array(y_pred).astype(int)

submission_df = pd.DataFrame({
    'id': df_test['id'],
    'target': y_pred
})
submission_df.to_csv('submission_new.csv', index=False)